## Introduction to Model Development with Spark

This will be the first of three parts of a bootcamp on Model Development with [MLlib](https://spark.apache.org/docs/latest/ml-guide.html), Spark’s machine learning (ML) library.  You will gain hands-on experience with essential steps of a model development using MLlib, which has has the goal to make machine learning scalable and easy. 

At a high level, MLlib provides tools such as:
- ML Algorithms: common learning algorithms such as classification, regression, clustering
- Featurization: feature extraction, transformation, dimensionality reduction, and selection
- Pipelines: tools for constructing, evaluating, and tuning ML Pipelines
- Persistence: saving and load algorithms, models, and Pipelines
- Utilities: linear algebra, statistics, data handling, etc.

In this lab, we will cover:
- Splitting of data for training and testing
- Applying Transformers to data frames
- Fitting Estimators to our data
- Creating and executing a ML Pipeline
- Model Evaluation.

We will learn how to use these features and tools by solving a common task in Natural Language Processing (NLP): Sentiment Analysis.  Our dataset contains roughly 50,000 movie reviews from the internet movie database (IMDB).  Each entry is a movie review written in the English language, as well as the author's rating of the movie on a scale from 1 to 10.  Based on the text of the review, we want to predict if the rating is "positive" or "negative".

Other applications of Sentiment Analysis:
- Detecting negative affect in customers who are calling an automated customer hotline
- Agreggating reviews of retail products into an overall rating for each product

## Initialize Notebook Environment

As usual we start by mounting our data and setting up our environment.

In [3]:
%run "../includes/mnt_blob"

In [4]:
%run "../includes/setup_env"

In [5]:
reviewsDF = spark.read.parquet("/mnt/data/imdb/imdb_ratings_50k.parquet")
reviewsDF.createOrReplaceTempView("reviews")
display(reviewsDF)

### Hands-on lab

What does the distribution of scores look like?

HINT: Use `count()`

In [8]:
%sql

-- Put your solution here

In [9]:
%sql

-- Maximize this cell to see the solution

select rating, count(*) from reviews group by rating order by rating;

### End of Lab

The authors of this dataset have removed the "neutral" ratings, which they defined as a rating of 5 or 6.

## Train-Test Split

We'll split our data into training and test samples. We will use 80% for training, and the remaining 20% for testing. We set a seed to reproduce the same results (i.e. if you re-run this notebook, you'll get the same results both times).

In [13]:
(trainDF, testDF) = reviewsDF.randomSplit([0.8, 0.2], seed=42)
trainDF.cache()
testDF.cache()

Let's determine our baseline accuracy.

In [15]:
positiveRatings = trainDF.filter("rating >= 5").count()
totalRatings = trainDF.count()

print("Baseline accuracy: {0:.2f}%".format(positiveRatings/totalRatings*100))

## Transformers

A transformer takes in a DataFrame, and returns a new DataFrame with one or more columns appended to it. They implement a `.transform()` method.

Let's get started by using [RegexTokenizer](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.RegexTokenizer) to convert our review string into a list of tokens.

In [18]:
from pyspark.ml.feature import RegexTokenizer

tokenizer = (RegexTokenizer()
            .setInputCol("review")
            .setOutputCol("tokens")
            .setPattern("\\W+"))

tokenizedDF = tokenizer.transform(reviewsDF)

display(tokenizedDF.limit(5)) # Look at a few tokenized reviews

There are a lot of words that do not contain much information about the sentiment of the review (e.g. `the`, `a`, etc.). Let's remove these uninformative words using [StopWordsRemover](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.StopWordsRemover).

In [20]:
from pyspark.ml.feature import StopWordsRemover

remover = (StopWordsRemover()
          .setInputCol("tokens")
          .setOutputCol("stopWordFree"))

removedStopWordsDF = remover.transform(tokenizedDF)
display(removedStopWordsDF.limit(5)) # Look at a few tokenized reviews without stop words

### hands-on lab

Where do the stop words actually come from? Spark includes a small English list as a default, which we're implicitly using here.

Look into the Spark documentation, to find out how you can get a list of the stop words.

In [22]:
# your solution goes into this cell
stopwords = []
stopwords # this prints the stopwords.

In [23]:
# this gives you a list of the stopwords                                                                                                                                                                  

stopWords = remover.getStopWords()
stopWords

Now try to remove additional stop words. For example, let's remove each occurrence of the `br` from the reviews.

In [25]:
# replace this with your solution
removedStopWordsDF = remover.transform(tokenizedDF)

In [26]:
# these two lines will remove "br" and all other defined stop words
remover.setStopWords(["br", "nbsp"] + stopWords)
removedStopWordsDF = remover.transform(tokenizedDF)

In [27]:
display(removedStopWordsDF.limit(5)) # Look at a few tokenized reviews without stop words

## Estimators

Estimators take in a DataFrame, and return a model (a Transformer). They implement a `.fit()` method.

Let's apply a [CountVectorizer](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.CountVectorizer) model to convert our tokens into a vocabulary.

In [30]:
from pyspark.ml.feature import CountVectorizer

counts = (CountVectorizer()
          .setInputCol("stopWordFree")
          .setOutputCol("features")
          .setVocabSize(1000))

countModel = counts.fit(removedStopWordsDF) # It's a model

__Now let's adjust the label (target) values__

We want to group the reviews into "positive" or "negative" sentiment. So all of the star "levels" need to be collapsed into one of two groups. To accomplish this, we will use [Binarizer](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.feature.Binarizer).

In [32]:
from pyspark.ml.feature import Binarizer

binarizer = (Binarizer()
            .setInputCol("rating")
            .setOutputCol("label")
            .setThreshold(5.0))

Now we are going to use a Decision Tree model to fit to our dataset.

In [34]:
from pyspark.ml.classification import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

## Pipeline

Let's put all of these stages into a [Pipeline](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.Pipeline). This way, you don't have to remember all of the different steps you applied to the training set, and then apply the same steps to the test dataset. The pipeline takes care of that for you!

In [36]:
from pyspark.ml import Pipeline

pipeline = Pipeline().setStages([tokenizer, remover, counts, binarizer, dtc])
pipelineModel = pipeline.fit(trainDF)

We can extract the stages from our Pipeline, such as the Decision Tree model.

In [38]:
decisionTree = pipelineModel.stages[-1]
print(decisionTree.toDebugString)

### Hands-on lab (optional!)

Please skip this hands-on lab the first time you go through this notebook.

After you fitted your first pipeline that included a step with the `CountVectorizer`, you could try to improve the performance of you model, by substituting the `CountVectorizer` with `Word2Vec`, which creates a more sophisticated semantic representation than simply counting the frequency of words in each review. Make sure to also update the pipeline definition.

In [40]:
from pyspark.ml.feature import Word2Vec

# word2Vec = TODO: define you word2vec estimator here

# TODO: update the below pipeline definition, to make sure it uses word2Vec
pipeline = Pipeline().setStages([tokenizer, remover, counts, binarizer, dtc])
pipelineModel = pipeline.fit(trainDF)

When you are done defining your word2vec estimator and updating your pipeline. Run *all* the below cells again, and see whether the performance of your model has changed.

### End of (optional) hands-on lab.

Let's save the pipeline model.

In [43]:
fileName = userhome + "/tmp/DT_Pipeline"
pipelineModel.write().overwrite().save(fileName)

Now let's load the `PipelineModel` back in.

**Note**: You need to know what type of model you're loading in.

In [45]:
from pyspark.ml import PipelineModel
# Load saved model
savedPipelineModel = PipelineModel.load(fileName)

In [46]:
resultDF = savedPipelineModel.transform(testDF)

## Evaluate

We are going to use [MultiClassClassificationEvaluator](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator)  to evaluate our predictions (we are using MultiClass because the BinaryClassificationEvaluator does not support accuracy as a metric).

In [48]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy: %(result)s" % {"result": evaluator.evaluate(resultDF)})

#### Confusion Matrix

Let's see if we had more False Positive or False Negatives.

In [50]:
display(resultDF.groupBy("label", "prediction").count())

Later, we will see how to apply this pipeline to streaming data!

-sandbox
&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>